<a href="https://colab.research.google.com/github/multimodalart/majesty-diffusion/blob/main/latent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latent Majesty Diffusion v1.6
#### Formerly known as Princess Generator
##### Access our [Majestic Guide](https://multimodal.art/majesty-diffusion) (_under construction_), our [GitHub](https://github.com/multimodalart/majesty-diffusion), join our community on [Discord](https://discord.gg/yNBtQBEDfZ) or reach out via [@multimodalart on Twitter](https://twitter.com/multimodalart))
\
 
---
\


#### CLIP Guided Latent Diffusion by [dango233](https://github.com/Dango233/) and [apolinario (@multimodalart)](https://twitter.com/multimodalart). 
The LAION-400M-trained model and the modified inference code are from [CompVis Latent Diffusion](https://github.com/CompVis/latent-diffusion). The guided-diffusion method is modified by Dango233 based on [Katherine Crowson](https://twitter.com/RiversHaveWings)'s guided diffusion notebook. multimodalart savable settings, MMC and assembled the Colab. Check the complete list on our GitHub. Some functions and methods are from various code masters (nsheppard, DanielRussRuss and others)


## Changelog


In [ ]:
#@markdown Release: 1.2 (prior versions were Princess Generator and you can check [GitHub out for that](https://github.com/multimodalart/majesty-diffusion/))

#@markdown Changelog: 1.3 - better upscaler (learn how to use it on our [Majestic Guide](https://multimodal.art/majesty-diffusion))

#@markdown Changelog: 1.4 - better defaults, added OpenCLIP ViT-L/14 LAION-400M, fix CLOOB, adds modified dynamic thresholding, removes latent upscaler (was broken), adds RGB upscaler 

#@markdown Changelog 1.5 - even better defaults, better dynamic thresholidng, fixes range scale, adds var and mean scales, adds the possibility of blurring cuts

#@markdown Changelog 1.6 - ViT-L conditioning for latenet diffusion, adds noising and scaling during advanced scheduling phases, fixes linear ETA, adss LAION models

## Save model and outputs on Google Drive? 

In [ ]:
#@markdown Enable saving outputs to Google Drive to save your creations at AI/models
save_outputs_to_google_drive = True #@param {type:"boolean"}
#@markdown Enable saving models to Google Drive to avoid downloading the model every Colab instance
save_models_to_google_drive = True #@param {type:"boolean"}

if save_outputs_to_google_drive or save_models_to_google_drive:
    from google.colab import drive
    try:
      drive.mount('/content/gdrive')
    except:
      save_outputs_to_google_drive = False
      save_models_to_google_drive = False

model_path = "/content/gdrive/MyDrive/AI/models" if save_models_to_google_drive else "/content/"
outputs_path = "/content/gdrive/MyDrive/AI/latent_majesty_diffusion" if save_outputs_to_google_drive else "/content/outputs"
!mkdir -p $model_path
!mkdir -p $outputs_path
print(f"Model will be stored at {model_path}")
print(f"Outputs will be saved to {outputs_path}")

#If you want to run it locally change it to true
is_local = False
skip_installs = False
if(is_local):
  model_path = "/choose/your/local/model/path"
  outputs_path = "/choose/your/local/outputs/path"
  skip_installs = True

In [ ]:
#@title Model settings
#@markdown The `original` model is the model trained by CompVis in the LAION-400M dataset
#@markdown <br>The `finetuned` model is a finetune of the `original` model [by Jack000](https://github.com/Jack000/glid-3-xl) that generates less watermarks, but is a bit worse in text synthesis. Colab Free does not have enough run for the finetuned (for now)
#@markdown <br>The `ongo` and `erlich` models are models [fine-tuned by LAION](https://github.com/LAION-AI/ldm-finetune)on art (ongo) and erlich (logos) 
latent_diffusion_model = 'finetuned' #@param ["original", "finetuned", "ongo (fine tuned in paintings)", "erlich (fine tuned in logos)"]

# Setup stuff

In [ ]:
#@title Installation
if(not skip_installs):
    import subprocess
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    cards_requiring_downgrade = ["Tesla T4", "V100"]
    if any(cardstr in nvidiasmi_output for cardstr in cards_requiring_downgrade):
        downgrade_pytorch_result = subprocess.run(['pip', 'install', 'torch==1.10.2', 'torchvision==0.11.3', '-q'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    import sys
    sys.path.append(".")
    !git clone https://github.com/multimodalart/latent-diffusion --branch 1.6
    !git clone https://github.com/CompVis/taming-transformers
    !git clone https://github.com/TencentARC/GFPGAN
    !git lfs clone https://huggingface.co/datasets/multimodalart/latent-majesty-diffusion-settings
    !git clone https://github.com/NightmareAI/majesty-diffusion --branch 1.6-cli
    !git lfs clone https://github.com/LAION-AI/aesthetic-predictor
    !pip install -e ./taming-transformers
    !pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops
    !pip install transformers
    !pip install dotmap
    !pip install resize-right
    !pip install piq
    !pip install lpips
    !pip install basicsr
    !pip install facexlib
    !pip install realesrgan

    sys.path.append('./taming-transformers')
    from taming.models import vqgan
    from subprocess import Popen, PIPE
    try:
        import mmc
    except:
        # install mmc
        !git clone https://github.com/apolinario/Multi-Modal-Comparators --branch gradient_checkpointing
        !pip install poetry
        !cd Multi-Modal-Comparators; poetry build
        !cd Multi-Modal-Comparators; pip install dist/mmc*.whl
        
        # optional final step:
        #poe napm_installs
        !python Multi-Modal-Comparators/src/mmc/napm_installs/__init__.py
    # suppress mmc warmup outputs
    import mmc.loaders

Now, download the checkpoint (~5.7 GB). This will usually take 3-6 minutes.

In [ ]:
#@title Download models
import sys
sys.path.append('./majesty-diffusion')
import majesty
majesty.model_path = model_path
majesty.download_models()


Let's also check what type of GPU we've got.

In [ ]:
!nvidia-smi

In [ ]:
#@title Import stuff
import argparse, os, sys, glob
sys.path.append('./majesty-diffusion')
import majesty
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm.auto import tqdm, trange
tqdm_auto_model = __import__("tqdm.auto", fromlist=[None]) 
sys.modules['tqdm'] = tqdm_auto_model
from einops import rearrange
from torchvision.utils import make_grid
import transformers
import gc
sys.path.append('./latent-diffusion')
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.modules.diffusionmodules.util import noise_like, make_ddim_sampling_parameters
import tensorflow as tf
from dotmap import DotMap
import ipywidgets as widgets
from math import pi

from subprocess import Popen, PIPE

from dataclasses import dataclass
from functools import partial
import gc
import io
import math
import sys
import random
from piq import brisque
from itertools import product
from IPython import display
import lpips
from PIL import Image, ImageOps
import requests
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models
from torchvision import transforms
from torchvision import transforms as T
from torchvision.transforms import functional as TF
from numpy import nan
from threading import Thread
import time
import re
import base64

#sys.path.append('../CLIP')
#Resizeright for better gradient when resizing
#sys.path.append('../ResizeRight/')
#sys.path.append('../cloob-training/')

from resize_right import resize

import clip
#from cloob_training import model_pt, pretrained

#pretrained.list_configs()
from torch.utils.tensorboard import SummaryWriter


In [ ]:
#@title Load the model
majesty.model_path = model_path
majesty.outputs_path = outputs_path
torch.backends.cudnn.benchmark = True
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
majesty.device = device

config = OmegaConf.load("./latent-diffusion/configs/latent-diffusion/txt2img-1p4B-eval.yaml")  # TODO: Optionally download from same location as ckpt and chnage this logic
model = majesty.load_model_from_config(config, f"{model_path}/latent_diffusion_txt2img_f8_large.ckpt",False, latent_diffusion_model)  # TODO: check path
majesty.model = model.half().eval().to(device)
#if(latent_diffusion_model == "finetuned"):
#  model.model = model.model.half().eval().to(device)
majesty.load_lpips_model()
majesty.load_aesthetic_model()

# Run!

#### Perceptors (Choose your CLIP and CLIP-like models) 
Be careful if you don't pay for Colab Pro selecting more CLIPs might make you go out of memory. If you do have Pro, try adding ViT-L14 to your mix

In [ ]:
#@title Choose your perceptor models

# suppress mmc warmup outputs
import mmc.loaders
clip_load_list = []
#@markdown #### Open AI CLIP models
ViT_B32 = False #@param {type:"boolean"}
ViT_B16 = True #@param {type:"boolean"}
ViT_L14 = True #@param {type:"boolean"}
ViT_L14_336px = False #@param {type:"boolean"}
#RN101 = False #@param {type:"boolean"}
#RN50 = False #@param {type:"boolean"}
RN50x4 = False #@param {type:"boolean"}
RN50x16 = False #@param {type:"boolean"}
RN50x64 = False #@param {type:"boolean"}

#@markdown #### OpenCLIP models
ViT_B16_plus = False #@param {type: "boolean"}
ViT_B32_laion2b = True #@param {type: "boolean"}
ViT_L14_laion = False #@param {type:"boolean"}

#@markdown #### Multilangual CLIP models 
clip_farsi = False #@param {type: "boolean"}
clip_korean = False #@param {type: "boolean"}

#@markdown #### CLOOB models
cloob_ViT_B16 = False #@param {type: "boolean"}

# @markdown Load even more CLIP and CLIP-like models (from [Multi-Modal-Comparators](https://github.com/dmarx/Multi-Modal-Comparators))
model1 = "" # @param ["[clip - mlfoundations - RN50--openai]","[clip - mlfoundations - RN101--openai]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model2 = "" # @param ["[clip - mlfoundations - RN50--openai]","[clip - mlfoundations - RN101--openai]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}
model3 = "" # @param ["[clip - openai - RN50]","[clip - openai - RN101]","[clip - mlfoundations - RN50--yfcc15m]","[clip - mlfoundations - RN50--cc12m]","[clip - mlfoundations - RN50-quickgelu--yfcc15m]","[clip - mlfoundations - RN50-quickgelu--cc12m]","[clip - mlfoundations - RN101--yfcc15m]","[clip - mlfoundations - RN101-quickgelu--yfcc15m]","[clip - mlfoundations - ViT-B-32--laion400m_e31]","[clip - mlfoundations - ViT-B-32--laion400m_e32]","[clip - mlfoundations - ViT-B-32--laion400m_avg]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e31]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_e32]","[clip - mlfoundations - ViT-B-32-quickgelu--laion400m_avg]","[clip - mlfoundations - ViT-B-16--laion400m_e31]","[clip - mlfoundations - ViT-B-16--laion400m_e32]","[clip - sbert - ViT-B-32-multilingual-v1]","[clip - facebookresearch - clip_small_25ep]","[simclr - facebookresearch - simclr_small_25ep]","[slip - facebookresearch - slip_small_25ep]","[slip - facebookresearch - slip_small_50ep]","[slip - facebookresearch - slip_small_100ep]","[clip - facebookresearch - clip_base_25ep]","[simclr - facebookresearch - simclr_base_25ep]","[slip - facebookresearch - slip_base_25ep]","[slip - facebookresearch - slip_base_50ep]","[slip - facebookresearch - slip_base_100ep]","[clip - facebookresearch - clip_large_25ep]","[simclr - facebookresearch - simclr_large_25ep]","[slip - facebookresearch - slip_large_25ep]","[slip - facebookresearch - slip_large_50ep]","[slip - facebookresearch - slip_large_100ep]","[clip - facebookresearch - clip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc3m_40ep]","[slip - facebookresearch - slip_base_cc12m_35ep]","[clip - facebookresearch - clip_base_cc12m_35ep]"] {allow-input: true}

if ViT_B32: 
  clip_load_list.append("[clip - mlfoundations - ViT-B-32--openai]")
if ViT_B16: 
  clip_load_list.append("[clip - mlfoundations - ViT-B-16--openai]")
if ViT_L14: 
  clip_load_list.append("[clip - mlfoundations - ViT-L-14--openai]")
if RN50x4: 
  clip_load_list.append("[clip - mlfoundations - RN50x4--openai]")
if RN50x64: 
  clip_load_list.append("[clip - mlfoundations - RN50x64--openai]")
if RN50x16: 
  clip_load_list.append("[clip - mlfoundations - RN50x16--openai]")
if ViT_L14_laion: 
  clip_load_list.append("[clip - mlfoundations - ViT-L-14--laion400m_e32]")
if ViT_L14_336px:
  clip_load_list.append("[clip - mlfoundations - ViT-L-14-336--openai]")
if ViT_B16_plus:
  clip_load_list.append("[clip - mlfoundations - ViT-B-16-plus-240--laion400m_e32]")
if ViT_B32_laion2b:
  clip_load_list.append("[clip - mlfoundations - ViT-B-32--laion2b_e16]")
if clip_farsi:
  clip_load_list.append("[clip - sajjjadayobi - clipfa]")
if clip_korean:
  clip_load_list.append("[clip - navervision - kelip_ViT-B/32]")
if cloob_ViT_B16:
  clip_load_list.append("[cloob - crowsonkb - cloob_laion_400m_vit_b_16_32_epochs]")

if model1:
  clip_load_list.append(model1)
if model2:
  clip_load_list.append(model2)
if model3:
  clip_load_list.append(model3)


majesty.clip_load_list = clip_load_list
torch.cuda.empty_cache()
gc.collect()

#### Advanced settings for the generation
##### Access [our guide](https://multimodal.art/majesty-diffusion)  

In [ ]:
opt = DotMap()

#Change it to false to not use CLIP Guidance at all 
majesty.use_cond_fn = True

#Custom cut schedules and super-resolution. Check out the guide on how to use it a https://multimodal.art/majestydiffusion
majesty.custom_schedule_setting = [
 [50,1000,8],
 "gfpgan:1.5","scale:.9","noise:.55",
 [50,200,5],
]
              
#Cut settings
majesty.clamp_index = [2.1,1.6] #linear variation of the index for clamping the gradient  
majesty.cut_overview = [8]*500 + [4]*500
majesty.cut_innercut = [0]*500 + [4]*500
majesty.cut_ic_pow = .2
majesty.cut_icgray_p = [.1]*300+[0]*1000
majesty.cutn_batches = 1
majesty.cut_blur_n = [0]*300 + [0]*1000
majesty.cut_blur_kernel = 3
majesty.range_index =  [0]*200+ [5e4]*400 + [0]*1000
majesty.var_index = [2]*300+[0]*700
majesty.var_range = 0.5
majesty.mean_index = [0]*400+[0]*600
majesty.mean_range = 0.75
majesty.active_function = "softsign" # function to manipulate the gradient - help things to stablize
majesty.ths_method = "clamp"
majesty.tv_scales = [600]*1+[50]*1 +[0]*2

#If you uncomment next line you can schedule the CLIP guidance across the steps. Otherwise the clip_guidance_scale basic setting will be used
#clip_guidance_schedule = [10000]*300 + [500]*700

majesty.symmetric_loss_scale = 0 #Apply symmetric loss

#Latent Diffusion Advanced Settings
majesty.scale_div = 1 # Use when latent upscale to correct satuation problem
majesty.opt_mag_mul = 20 #Magnify grad before clamping
#PLMS Currently not working, working on a fix
opt.plms = False #Experimental. It works but does not lookg good
majesty.opt_ddim_eta, majesty.opt_eta_end = [1.3,1.1] # linear variation of eta
majesty.opt_temperature = .98

#Grad advanced settings
majesty.grad_center = False
majesty.grad_scale= 0.25 #Lower value result in more coherent and detailed result, higher value makes it focus on more dominent concept

#Restraints the model from explodign despite larger clamp
majesty.score_modifier = True
majesty.threshold_percentile = .85
majesty.threshold = 1
majesty.score_corrector_setting = ["latent", ""]

#Init image advanced settings
majesty.init_rotate, majesty.mask_rotate=[False, False]
majesty.init_magnitude = 0.18215

#More settings
majesty.RGB_min, majesty.RGB_max = [-0.95,0.95]
majesty.padargs = {"mode":"constant", "value": -1} #How to pad the image with cut_overview
majesty.flip_aug=False
majesty.cutout_debug = False
majesty.opt.outdir = outputs_path

#Experimental aesthetic embeddings, work only with OpenAI ViT-B/32 and ViT-L/14
majesty.experimental_aesthetic_embeddings = True
#How much you want this to influence your result
majesty.experimental_aesthetic_embeddings_weight = 0.3
#9 are good aesthetic embeddings, 0 are bad ones
majesty.experimental_aesthetic_embeddings_score = 8

# For fun dont change except if you really know what your are doing
majesty.grad_blur = False
majesty.compress_steps = 200
majesty.compress_factor = 0.1
majesty.punish_steps = 200
majesty.punish_factor = 0.5

### Prompts
The main prompt is the CLIP prompt. The Latent Prompts usually help with style and composition.

In [ ]:
#Amp up your prompt game with prompt engineering, check out this guide: https://matthewmcateer.me/blog/clip-prompt-engineering/
#Prompt for CLIP Guidance
clip_prompts =["The portrait of a Majestic Princess, trending on artstation"]  

#Prompt for Latent Diffusion
latent_prompts = ["The portrait of a Majestic Princess, trending on artstation"]  

#Negative prompts for Latent Diffusion
latent_negatives = [""]

majesty.image_prompts = []

### Diffuse!

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#@markdown ### Basic settings 
#@markdown We're still figuring out default settings. Experiment and <a href="https://huggingface.co/datasets/multimodalart/latent-majesty-diffusion-settings">share your settings with us</a>
majesty.width =  256#@param{type: 'integer'}
majesty.height =  256#@param{type: 'integer'}
#@markdown The `latent_diffusion_guidance_scale` will determine how much the `latent_prompts` affect the image. Lower help with text interpretation, higher help with composition. Try values between 0-15. If you see too much text, lower it  
majesty.latent_diffusion_guidance_scale = 12 #@param {type:"number"}
#@markdown The `clamp_index` will determine how much of the `clip_prompts` affect the image, it is a linear scale that will decrease from the first to the second value. Try values between 3-1
majesty.clamp_index = [2.4, 2.1] #@param{type: 'raw'}
majesty.clip_guidance_scale =  16000#@param{type: 'integer'}
majesty.how_many_batches = 1 #@param{type: 'integer'}
majesty.aesthetic_loss_scale = 400 #@param{type: 'integer'}
majesty.augment_cuts=True #@param{type:'boolean'}

#@markdown

#@markdown  ### Init image settings
#@markdown `init_image` requires the path of an image to use as init to the model
majesty.init_image = None #@param{type: 'string'}
if(majesty.init_image == '' or majesty.init_image == 'None'):
  majesty.init_image = None
#@markdown `starting_timestep`: How much noise do you want to add to your init image for it to then be difused by the model
majesty.starting_timestep = 0.9 #@param{type: 'number'}
#@markdown `init_mask` is a mask same width and height as the original image with the color black indicating where to inpaint
majesty.init_mask = None #@param{type: 'string'}
#@markdown `init_scale` controls how much the init image should influence the final result. Experiment with values around `1000`
majesty.init_scale = 1000 #@param{type: 'integer'}
majesty.init_brightness = 0.0 #@param{type: 'number'}
#  @markdown How much extra noise to add to the init image, independently from skipping timesteps (use it also if you are upscaling)
#majesty.init_noise = 0.57 #@param{type: 'number'}

#@markdown

#@markdown ### Custom saved settings
#@markdown If you choose custom saved settings, the settings set by the preset overrule some of your choices. You can still modify the settings not in the preset. <a href="https://github.com/multimodalart/majesty-diffusion/tree/main/latent_settings_library">Check what each preset modifies here</a>
majesty.custom_settings = 'path/to/settings.cfg' #@param{type:'string'}
settings_library = 'None (use settings defined above)' #@param ["None (use settings defined above)", "default", "defaults_v1_3", "dango233_princesses", "the_other_zippy_defaults", "makeitrad_defaults"]
if(settings_library != 'None (use settings defined above)'):
  custom_settings = f'latent-majesty-diffusion-settings/{settings_library}.cfg'

majesty.load_custom_settings()
majesty.config_init_image()

majesty.prompts = clip_prompts
majesty.opt.prompt = latent_prompts
majesty.opt.uc = latent_negatives
majesty.set_custom_schedules()
majesty.config_clip_guidance()
majesty.config_output_size()
majesty.config_options()

torch.cuda.empty_cache()
gc.collect()
majesty.generate_video = False
majesty.do_run()


### Save your own settings


In [ ]:

#@markdown ### Save current settings
#@markdown If you would like to save your current settings, uncheck `skip_saving` and run this cell. You will get a `custom_settings.cfg` file you can reuse and share. If you like your results, <a href="https://huggingface.co/datasets/multimodalart/latent-majesty-diffusion-settings">share your settings with us on the settings library</a>
skip_saving = True #@param{type:'boolean'}
if(not skip_saving):
  data = majesty.generate_settings_file(add_prompts=False, add_dimensions=True)
  text_file = open("custom_settings.cfg", "w")
  text_file.write(data)
  text_file.close()
  from google.colab import files
  files.download('custom_settings.cfg')
  print("Downloaded as custom_settings.cfg")

### Biases acknowledgment
Despite how impressive being able to turn text into image is, beware to the fact that this model may output content that reinforces or exarcbates societal biases. According to the <a href='https://arxiv.org/abs/2112.10752' target='_blank'>Latent Diffusion paper</a>:<i> \"Deep learning modules tend to reproduce or exacerbate biases that are already present in the data\"</i>. 

The model was trained on an unfiltered version the LAION-400M dataset, which scrapped non-curated image-text-pairs from the internet (the exception being the the removal of illegal content) and is meant to be used for research purposes, such as this one. <a href='https://laion.ai/laion-400-open-dataset/' target='_blank'>You can read more on LAION's website</a>